In [291]:
import MDAnalysis as md
import numpy as np
import re
from vermouth.forcefield import ForceField
from vermouth.gmx.itp_read import read_itp
import networkx as nx
import os
import subprocess
from itertools import groupby

import warnings
warnings.filterwarnings('ignore')

In [339]:
PCG_path = ''  #Define path to PCG/TS2CG instalation
PLM_path = ''  #Define path to PLM/TS2CG instalation
IL_name = 'MC3H'
sterol_name = 'CHOL'
vesicle_radius = 25 # nm

build_file = open('input.str' , 'w')
build_file.write(f'[Lipids List]\n')
build_file.write(f'Domain 0\n')
build_file.write(f'{IL_name}  0.25  0.25 0.45\n')
build_file.write(f'DSPC  0.1  0.1 0.45\n')
build_file.write(f'{sterol_name}  0.25  0.25 0.45\n')
build_file.write(f'End\n')
build_file.close()

In [295]:
def itp_to_forcefield(itp):
    """
    Returns a forcefield with all the molecule in the itp added.

    The itp should specify the path to the ITP file.
    """
    path = itp
    with open(path, 'r') as f:
        lines = f.readlines()
    forcefield = ForceField(name='custom')
    read_itp(lines, forcefield)
    return forcefield
    
def get_radius (x,y,z):
    # Calculate the centroid of the cylinder (center axis)
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    centroid_z = np.mean(z)

    # Calculate the distances from the centroid to each point
    distances = np.sqrt((x - centroid_x)**2 + (y - centroid_y)**2 + (z - centroid_z)**2)

    # Find the minimum distance, which represents the internal radius of the cylinder
    internal_radius = np.min(distances)

    return internal_radius

## Join both compartments

In [322]:
rna_comp = md.Universe('m.gro') #(RNA_comp)
rna_comp.atoms.positions = rna_comp.atoms.positions - rna_comp.atoms.center_of_mass()
#radius of RNA sphere, trying to add 40 angstorm to accont for the double bilayer
sel = rna_comp.select_atoms('name N1 NP ROH')
R = get_radius(sel.positions[:,0], sel.positions[:,1], sel.positions[:,2]) + 55 ## Take into account the monolayer

In [323]:
oil_comp = md.Universe('m2.gro') #(oil_comp)
oil_comp.atoms.positions = (oil_comp.atoms.positions - oil_comp.atoms.center_of_mass()) + R

In [324]:
sys = md.Merge(rna_comp.select_atoms('all'), oil_comp.select_atoms('all'))
sys.atoms.write('Core.pdb')

In [299]:
# Import the itps which are sub dependencies
ff_rna = itp_to_forcefield('ITP/1000_RNA.itp')
ff_lipids = itp_to_forcefield('ITP/martini_v3.0.0_phospholipids_v1.itp')
ff_lisbeth = itp_to_forcefield('ITP/MC3_KC2_DP_DT_LI5_LI2_LI10_BMHB.itp')
ff_sterols = itp_to_forcefield('ITP/martini_v3.0_sterols_v1.0.itp')
ff_solvents = itp_to_forcefield('ITP/martini_v3.0.0_solvents_v1_BMHB.itp')
ff_ions = itp_to_forcefield('ITP/martini_v3.0.0_ions_v1.itp')
# Combine all the blocks in one big dictionary (a block is a molecule in our case)
all_blocks = {}
for forcefield in [ff_rna, ff_lipids, ff_lisbeth, ff_sterols, ff_solvents, ff_ions]:
    for name, block in forcefield.blocks.items():
        if name in all_blocks.keys():
            print(f'A double definition was found for {name}, this often results in issues!')
        all_blocks[name] = block
#all_blocks['RNA'].nodes(data=True)

In [300]:
molecule_identifiers = {}
for molecule in all_blocks.keys():
    molecule_identifiers[molecule] = []
    for key, values in all_blocks[molecule].nodes(data=True):
        molecule_identifiers[molecule].append((values['resname'], values['atomname']))
#molecule_identifiers

In [301]:
# Read in the GRO file
universe = md.Universe('Core.pdb')
os.system('gmx editconf -f Core.pdb -d 5 -c -o Core.pdb')

                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx editconf -f Core.pdb -d 5 -c -o Core.pdb


Back Off! I just backed up Core.pdb to ./#Core.pdb.5#


Note that major changes are planned in future for editconf, to improve usability and utility.
Read 399561 atoms
Volume: 0.001 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size : 74.960 75.046 75.300 (nm)
    center      : 15.196 15.160 15.189 (nm)
    box vectors :  0.100  0.100  0.100 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       : 27.284 27.363 27.461 (nm)
new center      : 42.481 42.523 42.650 (nm)
new box vectors : 84.960 85.046 85.300 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :616342.31               (nm^3)

If the molecule rotates the actual distance will be smaller. You might want
to use a cubic box instead, or why not try a dodecahedron today?



GROMACS reminds you: "They Paint Their Faces So Differently From Ours" (Gogol Bordello)



0

In [302]:
# Start matching the resnames
# This is not a very generic approach, but it can be expanded. The issue
#  is that there is no true solution to this problem. For certain molecules
#  might be subgroups of other molecules, meaning it is to a certain degree
#  an ambigous problem. Here we map small segments over long segments for 
#  simplicity, this might or might not be a problem for you system.
#  This can largely be circumvented by making sure that all resnames are
#  unique for every molecule type.
matches = []
active_molecule = []
in_RNA = False
for residue in universe.residues:
    # Matching by residue name if the molecule
    #  is a residue.
    if residue.resname in all_blocks.keys():
        in_RNA = False
        matches.append(residue.resname)
    # The naming of IONS is weird as they are separated
    #  molecules which have the same resname
    elif residue.resname == 'ION':
        in_RNA = False
        matches.append(residue.atoms.names[0])
    # The RNA or any polymer is an issue for
    #  their residues are not molecules. Here
    #  I wrote a basic detection for RNA. But
    #  this might need to be changed for other
    #  polymers. I only match by length, this
    #  might be lazy but for now I think it 
    #  will work.
    elif residue.resname in ['RA', 'RA3', 'RA5', 'RG']:
        if in_RNA == False:
            in_RNA = len(residue.atoms)
        else:
            in_RNA += len(residue.atoms)
        if in_RNA == len(molecule_identifiers['RNA']):
            in_RNA = False
            matches.append('RNA')
    else:
        in_RNA = False
        print(f'UNKNOWN RESIDUE {residue.resname}')
        break

In [303]:
successive_elements = []
previous_match = None
counter = None
for match in matches:
    if match == previous_match:
        counter += 1
    else:
        if previous_match is not None:
            successive_elements.append((previous_match, counter))
        counter = 1
        previous_match = match
successive_elements.append((match, counter))

In [304]:
# Quick sanity check to see if the amount of atoms
#  based on the segment counting is indeed the equal
#  to the total amount of atoms in the PDB/GRO.
amount_of_atoms = 0
for element in successive_elements:
    amount_of_atoms += len(molecule_identifiers[element[0]])*element[1]
all_fine = amount_of_atoms == len(universe.atoms)
print(f'The amount of atoms based on the segments in correct: {all_fine}')

The amount of atoms based on the segments in correct: True


In [305]:
# Writing the mol_counting file which can be copied or imported in the top.
if all_fine:
    with open('Core_mol_counting.top', 'w') as f:
        for successive_element in successive_elements:
            f.write(f'{successive_element[0]}\t{successive_element[1]}\n')
    print('The mol_counting.top as been written succesfully.')
else:
    'No ouput has been generated as the amount of atoms does not match.'

The mol_counting.top as been written succesfully.


In [306]:
topol = open('Core.top', 'w')
            
topol.write(f'#include "ITP/martini_v3.0.0.itp"\n')
topol.write(f'#include "ITP/martini_v3.0_ffbonded.itp"\n')
topol.write(f'#include "ITP/1000_RNA.itp"\n')
topol.write(f'#include "ITP/martini_v3.0.0_phospholipids_v1.itp"\n')
topol.write(f'#include "ITP/MC3_KC2_DP_DT_LI5_LI2_LI10_BMHB.itp"\n')
topol.write(f'#include "ITP/martini_v3.0_sterols_v1.0.itp"\n')
topol.write(f'#include "ITP/martini_v3.0.0_solvents_v1.itp"\n')
topol.write(f'#include "ITP/martini_v3.0.0_ions_v1.itp"\n')
topol.write(f'[ system ]\n')
topol.write(f'LNP\n')
topol.write(f'\n')
topol.write(f'[ molecules ]\n')
topol.write(f'#include "Core_mol_counting.top"\n')
topol.close()

In [307]:
os.system('touch empty.mdp')
os.system('gmx grompp -f empty.mdp -c Core.pdb -p Core.top -o Core_fake.tpr')

p = subprocess.Popen('gmx trjconv -f Core.pdb -pbc whole -s Core_fake.tpr -o Core_fake_whole.pdb -conect'
                    , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
p.communicate('0\n')
p.wait()

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx grompp -f empty.mdp -c Core.pdb -p Core.top -o Core_fake.tpr


NOTE 1 [file empty.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.


NOTE 2 [file Core_mol_counting.top, line 7]:
  System has non-zero total charge: -1000.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




NOTE 3 [file Core_mol_counting.top, line 7]:
  For energy conservation with LINCS, lincs_iter should be 2 or larger.


Number of degrees of freedom in T-Coupling group rest is 986296.00
The integr

Setting the LD random seed to 2145772023

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'RNA'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'CL'

Excluding 1 bonded neighbours molecule type 'MC3'

Excluding 1 bonded neighbours molecule type 'CHOL'

Cleaning up constraints and constant bonded interactions with virtual sites

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are: 65700      Other residues
There are:  1000        RNA residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 30 Mb of data


                     :-) GROMACS - gmx trjconv, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx trjconv -f Core.pdb -pbc whole -s Core_fake.tpr -o Core_fake_whole.pdb -conect

Will write pdb: Protein data bank file
Reading file Core_fake.tpr, VERSION 2023.3 (single precision)
Reading file Core_fake.tpr, VERSION 2023.3 (single precision)
Group     0 (         System) has 399561 elements
Group     1 (          Other) has 387561 elements
Group     2 (           MC3H) has 89844 elements
Group     3 (           CHOL) has 96192 elements
Group     4 (              W) has 26038 elements
Group     5 (            ION) has  7487 elements
Group     6 (            MC3) has 168000 elements
Reading frame       0 time    0.000   b file
Precision of Core.pdb is 0.0001 (nm)

Back Off! I just ba

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'



GROMACS reminds you: "Nothing shocks me. I'm a scientist." (Harrison Ford as Indiana Jones)



0

### Minimize in vacuum

In [308]:
os.system('gmx grompp -f MDPs/min_vac.mdp -c Core_fake_whole.pdb -p Core.top -o Core_fake_whole.tpr')

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx grompp -f MDPs/min_vac.mdp -c Core_fake_whole.pdb -p Core.top -o Core_fake_whole.tpr


NOTE 1 [file Core_mol_counting.top, line 7]:
  System has non-zero total charge: -1000.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.



Number of degrees of freedom in T-Coupling group rest is 986296.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

NOTE 2 [file MDPs/min_vac.mdp]:
  You are using a plain Coulomb cut-off, which might produce artifacts.
  You might want to consider using PME electrostatics.



There we

Setting the LD random seed to 1568661450

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3H'

Excluding 1 bonded neighbours molecule type 'CHOL'

Excluding 1 bonded neighbours molecule type 'RNA'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'CL'

Excluding 1 bonded neighbours molecule type 'MC3'

Excluding 1 bonded neighbours molecule type 'CHOL'

Cleaning up constraints and constant bonded interactions with virtual sites

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are: 65700      Other residues
There are:  1000        RNA residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 31 Mb of data


0

In [309]:
os.system('gmx mdrun -deffnm Core_fake_whole_cubic -v -nt 10')

                      :-) GROMACS - gmx mdrun, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx mdrun -deffnm Core_fake_whole_cubic -v -nt 10


Back Off! I just backed up Core_fake_whole_cubic.log to ./#Core_fake_whole_cubic.log.4#
Reading file Core_fake_whole_cubic.tpr, VERSION 2023.3 (single precision)
Update groups can not be used for this system because an incompatible virtual site type is used

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


NOTE: The number of threads is not equal to the number of (logical) cpus
      and the -pin option is set to au

0

## Get Blender mesh

In [310]:
os.system('vmd -dispdev text -e tcl_script/render_stl.tcl -args Core.pdb test.stl')

rlwrap: Command not found.


Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 40 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 AVX512F AVX512CD HT 
Info) Free system memory: 64GB (34%)
Info) Creating CUDA device pool and initializing hardware...
Info) Detected 1 available CUDA accelerator::
Info) [0] NVIDIA T1000 8GB     14 SM_7.5 1.4 GHz, 7.8GB RAM SP32 KT AE3 ZC
Info) Dynamically loaded 3 plugins in directory:
Info) /usr/local/lib/vmd/plugins/LINUXAMD64/molfile
Core.pdb
test.stl
OptiXRenderer) ERROR: Failed to load OptiX shared library.
OptiXRenderer)        

0

In [311]:
os.system('/projects/DAMM/blender-4.0.2-linux-x64/blender --background --python remesh.py')

Import finished in 3.1482 sec.
Blender 4.0.2 (hash 9be62e85b727 built 2023-12-05 08:48:50)
Read prefs: "/home/mvalerio/.config/blender/4.0/config/userpref.blend"

Blender quit


0

## Make a bilayer for top and bottom instead and then a monolayer for the sizes. 

In [340]:
#mono 2
os.system(f"{PLM_path} -TSfile out_test.tsi -rescalefactor {vesicle_radius*2} {vesicle_radius*2} {vesicle_radius*2} -smooth -monolayer 1 -o mono_layer2")
os.system(f"{PCG_path} -str input.str -LLIB Martini3.LIB -defout mono2 -dts mono_layer2")

---> Note: the tsi file version is 1.1
We will increase the number of the available points by 4^3

****************** CG Membrane builder ******************   
******************  Version:  1.1 ****************** 
--> Generating molecule types from  input.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 146 lipid types 
--> no inclsuion file is provided, we will generate a random distribution of proteins if information is provided in STR file 
--> Note: the total monolayer area is 16369.7 
 We have placed the proteins, now is time to add the lipids 
 warrning: the total lipid percentage for domain 0 is not 1. It is (0.6) make sure you know what are you doing or use -renorm option 
 warrning: the total lipid percentage for domain 0 is not 1. It is (0.6) make sure you know what are you doing or use -renorm option 
 Number of the domains defined in the input file  1
***************************  we aim t

0

In [367]:
m2 = md.Universe('mono2.gro')
m1 = md.Universe('Bi-layer2.gro')
c  = md.Universe('Core.pdb')

univers = [m1, m2, c]

In [368]:
for u in univers:
    u.atoms.positions = u.atoms.positions - u.atoms.center_of_geometry()

In [369]:
c.atoms.positions = c.atoms.positions + (vesicle_radius)

In [370]:
combined = md.Merge(m2.atoms, c.atoms)

In [371]:
combined.atoms.write('coated_test.pdb')

In [372]:
R

277.75244140625